# libs & configs

In [1]:
from rapidfuzz import process, fuzz
import unidecode
import re
import calendar
import pandas as pd
import xlsxwriter
from openpyxl import load_workbook
import os
import sys
from pandas import to_datetime as pdDatetime
from datetime import datetime
from dateutil.relativedelta import relativedelta
from pandas import to_numeric as pdNumeric
from collections import defaultdict


In [2]:
# pega o diretorio acima de noteboosks, ou seja o do projeto principal
# current_dir = os.path.dirname(os.path.abspath(__file__)) - # isso nao funciona no jupyter , pois o __file__ e exclusivo de scrpit py
current_dir = os.getcwd()

# muda o diretorio para o projeto --
# isso é necessario para que o python consiga importar os modulos do projeto
# caso contrario, ele nao consegue achar os modulos
project_root = os.path.join(current_dir, '..')
sys.path.append(project_root)

In [3]:
from Scripts.functions import GetAno, mapear_vendedores_por_similaridade


# 00 - montando base de informaçoes

## Regioes 

In [21]:
# Criar os dados com abreviações
estados_regiões = {
    "Estado": [
        "Acre", "Alagoas", "Amapá", "Amazonas", "Bahia", "Ceará", "Distrito Federal",
        "Espírito Santo", "Goiás", "Maranhão", "Mato Grosso", "Mato Grosso do Sul",
        "Minas Gerais", "Pará", "Paraíba", "Paraná", "Pernambuco", "Piauí",
        "Rio de Janeiro", "Rio Grande do Norte", "Rio Grande do Sul", "Rondônia",
        "Roraima", "Santa Catarina", "São Paulo", "Sergipe", "Tocantins"
    ],
    "Cd Uf": [
        "AC", "AL", "AP", "AM", "BA", "CE", "DF",
        "ES", "GO", "MA", "MT", "MS",
        "MG", "PA", "PB", "PR", "PE", "PI",
        "RJ", "RN", "RS", "RO", "RR",
        "SC", "SP", "SE", "TO"
    ],
    "Região": [
        "Norte", "Nordeste", "Norte", "Norte", "Nordeste", "Nordeste", "Centro-Oeste",
        "Sudeste", "Centro-Oeste", "Nordeste", "Centro-Oeste", "Centro-Oeste",
        "Sudeste", "Norte", "Nordeste", "Sul", "Nordeste", "Nordeste",
        "Sudeste", "Nordeste", "Sul", "Norte", "Norte",
        "Sul", "Sudeste", "Nordeste", "Norte"
    ]
}

## clientes °

In [22]:
# -- 1 lendo arquivos
df_clients_bling = pd.read_excel(r"..\Data\01-01-23_08-07-25\sh-clientes_bling.xlsx")
df_clients_infosoft = pd.read_excel(r"..\Data\01-01-23_08-07-25\sh-clientes_infosoft.xlsx")

# -- 2 - padroniza os nomes para maior facilidade
df_clients_infosoft.rename(columns={'Cgc Cpf':'Clie Cgc Cpf'}, 
                                inplace=True)
df_clients_bling.rename(columns={'CPF/CNPJ': 'Clie Cgc Cpf',
                                  'Nome':'Descrição', 
                                  'Endereço': 'Endereco',
                                  'UF':'Cd Uf', 
                                  'Cidade':'Nm Cidade', 
                                  'Bairro':'Bairro',
                                'Cliente desde':'Dt Cadastro', 
                                'Email':'E Mail'
                                }, inplace=True)

# -- 3 - faz o full join 
df_clientes_merge = pd.merge(df_clients_infosoft,df_clients_bling,
                                  on='Clie Cgc Cpf',   # chave primaria
                                  how='outer',    # full outer join no pandas       
                                  suffixes=('_infosoft','_bling')) # renomeias o sufixo das colunas (col1_df1,col2_df2...)

# -- 4 - filtrando para todos os clients ou seja '''full join com priorização do df1'''
cols = ['Descrição', 'Endereco', 'Cd Uf','Nm Cidade', 'Bairro', 'Dt Cadastro','E Mail'] # colunas que quero manter (na esquerda)
for col in cols: 
    df_clientes_merge[col] = df_clientes_merge[f'{col}_infosoft'].combine_first(df_clientes_merge[f'{col}_bling']) # where quando a esquerda for vazia colca conteudo da direita, colca tudo na nova coluna 
    
df_clientes_masquerade = df_clientes_merge[['Clie Cgc Cpf'] + cols] # atribui a df final o conteudo completo nas colunas novas sem esquecer da chave primeira



## nfs ~


In [84]:
'''
corrigir cod vendedores bling
nr de pedido

'''


# -- 1 - le os arquivos
df_nfs_bling = pd.read_excel(r"..\Data\01-01-23_08-07-25\sh-nfs_bling.xlsx")
df_nfs_infosoft = pd.read_excel(r"..\Data\01-01-23_08-07-25\sh-nfs_infosoft.xlsx")

# -- 2 - padroniza as colunas para evitar complicações
df_nfs_bling.rename(columns={'Data de emissão':'Dt. emissão', 
                             'Data de Saída/Entrada':'Dt Sai Ent', 
                             'Número':'Nr. nota', 
                             'CNPJ/CPF':'Clie Cgc Cpf',
                             'Código do vendedor':'Cd Vendedor', 
                             'Desconto':'Vl Total Desc', 
                             'Valor total líquido':'Vl Total Nota'
                             },inplace=True)

# -- 3 -  junta as duas df 
df_nfs_masquerade = pd.concat([df_nfs_bling,df_nfs_infosoft], axis=0)

# # -- 4 - arrumando vendedores !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# df_nfs_masquerade['Cd Vendedor'] = df_nfs_masquerade['Cd Vendedor'].fillna('').astype(str).str.strip()

# df_nfs_masquerade, logs = mapear_vendedores_por_similaridade(df_nfs_masquerade, df_vendedores_masquerade)


# -- 5 - padroniza os tipos e filtr
df_nfs_masquerade['Clie Cgc Cpf'] = (
    df_nfs_masquerade['Clie Cgc Cpf']
    .astype(str)
    .str.strip()                  # remove espaços antes/depois
    .str.replace(r'\s+', '', regex=True)  # remove espaços internos
)
ice = ['27.191.195/0001-50',
'18.848.204/0001-42',
'08.913.661/0001-10',
 '11.990.266/0001-45',
 '05.012.654/0001-59',
 '04.466.924/0001-39',
 '44.667.686/0001-44',
 '45.385.860/0001-29',
 '41.400.310/0001-80' ]
df_nfs_masquerade = df_nfs_masquerade[~df_nfs_masquerade['Clie Cgc Cpf'].isin(ice)]
df_nfs_masquerade['Dt Sai Ent'] = pdDatetime(df_nfs_masquerade['Dt Sai Ent'], dayfirst=True)
df_nfs_masquerade['Dt. emissão'] = pdDatetime(df_nfs_masquerade['Dt. emissão'], dayfirst=True)
df_nfs_masquerade['Vl Total Desc'] = df_nfs_masquerade['Vl Total Desc'].astype(str).str.replace(',','')
df_nfs_masquerade['Vl Total Nota'] = df_nfs_masquerade['Vl Total Nota'].astype(str).str.replace(',','')
df_nfs_masquerade['Vl Total Desc'] = pdNumeric(df_nfs_masquerade['Vl Total Desc'],errors='coerce')
df_nfs_masquerade['Vl Total Nota'] =  pdNumeric(df_nfs_masquerade['Vl Total Nota'],errors='coerce')
df_nfs_masquerade['Nr Pedido'] =  pdNumeric(df_nfs_masquerade['Nr Pedido'],errors='coerce')



## pedidos

In [85]:
df_pedidos_infosoft = pd.read_excel(r"../Data/01-01-23_08-07-25/sh-pedidos_infosoft.xlsx")
df_pedidos_bling = pd.read_excel(r'..\Data\sh-pedidos-bling.xlsx')

df_pedidos_bling.rename(columns={'CPF/CNPJ':'CNPJ - CPF', 
                                      'Valor total': 'Vl. total',
                                      'Data': 'Dt. pedido',
                                      'N煤mero do pedido': 'Nr. pedido'
                                      },inplace=True)

df_pedidos_masquerade = pd.concat([df_pedidos_infosoft,df_pedidos_bling], axis=0)

df_pedidos_masquerade.rename(columns={'CNPJ - CPF':'Clie Cgc Cpf', 
                                      'Vl. total': 'Vl venda',
                                      'Nr. pedido': 'Nr Pedido',
                                      'Dt. pedido': 'Dt. emissão'
                                      },inplace=True)

# -- 5 - padroniza os tipos e filtr
df_pedidos_masquerade['Clie Cgc Cpf'] = (
    df_pedidos_masquerade['Clie Cgc Cpf']
    .astype(str)
    .str.strip()                  # remove espaços antes/depois
    .str.replace(r'\s+', '', regex=True)  # remove espaços internos
)
ice = ['27.191.195/0001-50',
'18.848.204/0001-42',
'08.913.661/0001-10',
 '11.990.266/0001-45',
 '05.012.654/0001-59',
 '04.466.924/0001-39',
 '44.667.686/0001-44',
 '45.385.860/0001-29',
 '41.400.310/0001-80' ]
df_pedidos_masquerade = df_pedidos_masquerade[~df_pedidos_masquerade['Clie Cgc Cpf'].isin(ice)]

df_pedidos_masquerade['Nr Pedido'] = pdNumeric(df_pedidos_masquerade['Nr Pedido'])
df_pedidos_masquerade['Vl venda'] = df_pedidos_masquerade['Vl venda'].astype(str).str.replace(',','')
df_pedidos_masquerade['Vl venda'] = pdNumeric(df_pedidos_masquerade['Vl venda'])
df_pedidos_masquerade['Dt. emissão'] =  pd.to_datetime(df_pedidos_masquerade['Dt. emissão'], dayfirst=True, )

print(df_pedidos_masquerade['Dt. emissão'].sort_values())


26     2023-07-03
20     2023-07-03
5      2023-07-04
6      2023-07-04
23     2023-07-05
          ...    
1469   2025-07-08
1468   2025-07-08
1466   2025-07-08
1462   2025-07-08
1467   2025-07-08
Name: Dt. emissão, Length: 1524, dtype: datetime64[ns]


## produtos

## vendedores °

In [25]:
vendedores_pix = pd.read_excel('..\Data\Posição representantes 2023 - 2024.xlsx', sheet_name='PIX REPRESENTANTES', skiprows=1)
vendedores_pix.rename(columns={'NOME':'Nm Vendedor'}, inplace=True)
vendedores_pix = vendedores_pix[vendedores_pix['Nm Vendedor'].notna()]

df_vendedores_masquerade = pd.read_excel(r'..\Data\01-01-23_08-07-25\sh-vendedores_infosoft.xlsx')
df_vendedores_masquerade = df_vendedores_masquerade[~pd.isna(df_vendedores_masquerade['Nm Vendedor'])]

df_vendedores_masquerade['Cd Vendedor'] = df_vendedores_masquerade['Cd Vendedor'].astype(float) 

def limpar_nome(nome):
    nome = str(nome).upper()
    nome = unidecode.unidecode(nome)  # remove acentos
    nome = re.sub(r'\(.*?\)', '', nome)  # remove parênteses com cidade
    nome = re.sub(r'[^A-Z\s]', '', nome)  # remove pontuação
    nome = re.sub(r'\s+', ' ', nome).strip()  # normaliza espaços
    return nome

def corrigir_nomes_pix(df_pix, df_base, coluna='Nm Vendedor', score_minimo=85):
    """
    Faz fuzzy matching entre nomes de df_pix[coluna] e df_base[coluna],
    substitui os nomes do df_pix pelos correspondentes mais próximos da base
    e gera um log das alterações realizadas.

    Retorna:
    - df_pix com nomes corrigidos
    - DataFrame de log de substituições
    """
    nomes_pix = df_pix[coluna].dropna().unique()
    nomes_base = df_base[coluna].dropna().unique()

    nomes_pix_limpos = [limpar_nome(n) for n in nomes_pix]
    nomes_base_limpos = [limpar_nome(n) for n in nomes_base]

    mapa_base = dict(zip(nomes_base_limpos, nomes_base))
    mapa_pix_original = dict(zip(nomes_pix_limpos, nomes_pix))

    log_substituicoes = []

    # Criar cópia do DataFrame original
    df_corrigido = df_pix.copy()

    for nome_limpo in nomes_pix_limpos:
        match, score = process.extractOne(nome_limpo, nomes_base_limpos, scorer=fuzz.token_sort_ratio)

        if score >= score_minimo:
            nome_original = mapa_pix_original[nome_limpo]
            nome_corrigido = mapa_base[match]

            # Substituir no DataFrame
            df_corrigido.loc[df_corrigido[coluna] == nome_original, coluna] = nome_corrigido

            # Registrar log
            log_substituicoes.append({
                'Nome Original': nome_original,
                'Nome Corrigido': nome_corrigido,
                'Score': score
            })

    df_log = pd.DataFrame(log_substituicoes)
    return df_corrigido, df_log

vendedores_pix, log= corrigir_nomes_pix(vendedores_pix, df_vendedores_masquerade)

print(log)
df_vendedores_masquerade


ValueError: too many values to unpack (expected 2)


# 15 - faturamento completo mes ano, por "produto"



In [ ]:
''' mes / papelaria / mer_livre / sapatilha / eventos / modinha / puerilcultura / total 

total: / ....... /'''

##  16 - faturameno completo mes indivdual separado (modelo)



In [ ]:
'''
mes 
papelaria 
 mer_livre 
sapatilha 
eventos 
 modinha 
puerilcultura 
total 
'''

In [ ]:
# 1 preenchendo o modelo


# 05 - resumo de venda/faturamento - fechamento anual


In [86]:
''' mes-ano / vendas / faturado 
total / .../ '''

' mes-ano / vendas / faturado \ntotal / .../ '

In [87]:
# -- 1 - defines as bases de dados e trata certas colunas
df_nfs = df_nfs_masquerade[['Vl Total Nota','Dt. emissão']]
df_pedidos = df_pedidos_masquerade[['Vl venda','Dt. emissão']]

# -- 2 - pega os anos
anos = [2024, 2025]

# -- 3 - filtra com base nos anos e deixa penas o mes e o ano 
df_pedidos = df_pedidos[ # filtro
            (df_pedidos['Dt. emissão'].dt.year == anos[0]) |
            (df_pedidos['Dt. emissão'].dt.year == anos[1]) 
            ] 

df_nfs = df_nfs[ # filtro
            (df_nfs['Dt. emissão'].dt.year == anos[0]) |
            (df_nfs['Dt. emissão'].dt.year == anos[1]) 
            ] 

# -- 4 - trandorma em mes-ano
df_pedidos['Dt. emissão'] = df_pedidos['Dt. emissão'].dt.to_period('M')
df_nfs['Dt. emissão'] = df_nfs['Dt. emissão'].dt.to_period('M')

# -- 5 - Agrupa por mês e soma os valores
vendas_mensal = df_pedidos.groupby('Dt. emissão')['Vl venda'].sum().reset_index()
faturado_mensal = df_nfs.groupby('Dt. emissão')['Vl Total Nota'].sum().reset_index()

# -- 7 -  mergga os dois DataFrames pela coluna 'Dt. emissão'
df_raw_mes_ano = pd.merge(
    vendas_mensal,
    faturado_mensal,
    on='Dt. emissão',
    how='outer'  # inclui meses que têm apenas vendas ou apenas faturamento
).fillna(0)

# -- 8 - Ordena pelo mês-ano corretamente
df_raw_mes_ano = df_raw_mes_ano.sort_values('Dt. emissão')

# -- 9 - mud o nome ds colunas
df_raw_mes_ano.columns = ['mes-ano', 'venda', 'faturado']

total_2024_faturado = df_raw_mes_ano['faturado'].sum()
total_2024_venda = df_raw_mes_ano['venda'].sum()

print(f'2024 - total:\n faturado | venda \n {total_2024_faturado} | {total_2024_venda} ')
df_raw_mes_ano

2024 - total:
 faturado | venda 
 4712258.630000001 | 4869610.580000001 


,mes-ano,venda,faturado
0,2024-01,445156.41,423175.53
1,2024-02,187988.44,263371.18
2,2024-03,160810.80,164753.56
3,2024-04,234761.57,266658.30
4,2024-05,149648.48,158980.08
5,2024-06,203925.32,199868.63
6,2024-07,237206.66,219127.76
7,2024-08,558552.47,391515.80
8,2024-09,233862.85,328693.39
9,2024-10,312581.85,326704.35


In [88]:
df_raw_mes_ano.to_excel("hdk.xlsx")


# 09 - comissões por periodos

### extract + tranform (definindo dados e juntando dados )

In [85]:
# -- 1 - defino as coluas de nossa df bruta e trago as df necessrias para o relatorio 
colunas = ['Clie Cgc Cpf','Dt. emissão', 'Nr. nota', 'Cd Vendedor', 'Vl Total Nota', 'aliquota', 'comissao', 'total']
df_comission_raw = pd.DataFrame(columns=colunas)
df_vendedores = df_vendedores_masquerade.copy() # para pegar o vendedor
df_clientes = df_clientes_masquerade.copy() # para pegar o cliente
df_pedidos = df_pedidos_masquerade.copy() # para pegar os pedidos
df_nfs = df_nfs_masquerade[['Dt. emissão', 'Nr. nota','Nr Pedido', 'Clie Cgc Cpf', 'Cd Vendedor', 'Vl Total Nota']]

# -- 1 - atribuindo infos das nfs 
df_comission_raw = pd.concat([df_comission_raw,df_nfs], axis=0)

# -- 2 - defininido o periodo (mes e ano) e filtrando nossa df
ano = 2025
mes = 6
mes2 = 7 
df_comission_raw = df_comission_raw[(df_comission_raw['Dt. emissão'].dt.year == ano) & 
                (df_comission_raw['Dt. emissão'].dt.month == mes)]

# -- 3 - fazer um semi join (pega apegas nfs que tivrem com cd vendedor) - por preucação
df_comission_raw = df_comission_raw[df_comission_raw['Cd Vendedor'].isin(df_vendedores['Cd Vendedor'])] #  pandas não tem suporte direto para semi join o jeito a epanas filtrar oque e muito pratico!

# -- 4 - fazer merge para epgar o nome dos vendedores pelo cod, posteriormente pegar o supervisor e aliquota tambem 
df_comission_raw['aliquota'] = 7 # temporario
df_comission_raw = df_comission_raw.merge(df_vendedores[['Cd Vendedor', 'Nm Vendedor','Nm supervisor']], 
                             on='Cd Vendedor', how='left')

# -- 5 - fazer merge para pegar o nome do cliente (left join em duas colunas basicamente..)
df_comission_raw = df_comission_raw.merge(df_clientes[['Clie Cgc Cpf', 'Descrição']],
                      on='Clie Cgc Cpf', how='left')

# -- 6 - merge para pegar o valor do pedido
df_comission_raw = df_comission_raw.merge(df_pedidos[['Nr Pedido', 'Vl venda']],
                      on='Nr Pedido', how='left')

# -- 7 -  definindo comissão a ser recebida 
df_comission_raw['comissao'] = df_comission_raw['Vl Total Nota']*(df_comission_raw['aliquota']/100)

# -- 8 - definindo clientes novos
df_clientes_novos = df_clientes[df_clientes['Dt Cadastro'] > datetime(ano, mes, 1)]
df_clientes_novos['cliente_nv'] = 20

# -- 9 - fazendo merge para atribuir os clientes novos 
df_comission_raw = df_comission_raw.merge(df_clientes_novos[['Clie Cgc Cpf', 'cliente_nv']],
                      on='Clie Cgc Cpf', how='left', suffixes=('_raw','_new'))

# -- 10 - definindo comissão total 
df_comission_raw['total'] = df_comission_raw['cliente_nv'].fillna(0)+df_comission_raw['comissao'].fillna(0)

# -- 11 - atribuindo apenas informações necessarias
df_comission_raw = df_comission_raw[['Nm supervisor', 'Nm Vendedor','Dt. emissão', 'Nr. nota','Descrição','Vl venda', 'Vl Total Nota','aliquota','comissao','cliente_nv','total']]


# --- 12 - susbtituindo os supervisores vazios para sem supervisor 
df_comission_raw['Nm supervisor'] = df_comission_raw['Nm supervisor'].replace(['', ' ', 'nan', None], 'SEM SUPERVISOR')

# -- 13 - Ordenando para gerar linha linha excel
df_detalhes = df_comission_raw.sort_values(by=['Nm supervisor', 'Nm Vendedor', 'Dt. emissão']) 

#-- 14 - define os subtotais por vendedor 
subtotal_vendedor = df_comission_raw.groupby(['Nm supervisor', 'Nm Vendedor']).agg(
    total_venda=('Vl venda', 'sum'),
    total_faturado=('Vl Total Nota', 'sum'),
    total_comissao=('comissao', 'sum'),
    total_cliente_nv=('cliente_nv', 'sum'),
    total_total=('total', 'sum')
).reset_index()

# -- 15 - ve se o supervisor tambem e vendedor para na hora de gara os 3% do supervisor nao incluir se ele for eh_vendedor tiver true 
df_comission_raw['eh_vendedor'] = df_comission_raw['Nm supervisor'] == df_comission_raw['Nm Vendedor']

# -- 16 - define os Totais por supervisor (sem representante tambem vai estar aqui)
totais_supervisor = df_comission_raw.groupby('Nm supervisor').agg(
    total_comissao=('comissao', 'sum'),
    total_cliente_nv=('cliente_nv', 'sum')
)

# -- 17 - definindo o valor faturado para calculo dos 3%e os 3% ja nomeando para usar como coluna
faturamento_subordinados = df_comission_raw[df_comission_raw['eh_vendedor'] == False] \
                                        .groupby('Nm supervisor')['Vl Total Nota'].sum()

comissao_extra = faturamento_subordinados * 0.03
comissao_extra.name = 'comissao_extra_3pct'

# -- 18 - cria o resumo final dos supervisores  
resumo_supervisor = totais_supervisor.join(comissao_extra, how='left').fillna(0)
resumo_supervisor['comissao_final'] = (
    resumo_supervisor['total_comissao'] +
    resumo_supervisor['total_cliente_nv'] +
    resumo_supervisor['comissao_extra_3pct']
)

# -- GAMBIARRA - PEGA O PIX E FAVORECIdo 
df_detalhes = df_detalhes.merge(vendedores_pix[['Nm Vendedor','FAVORECIDO','CHAVE']], 
                                on='Nm Vendedor', how='left')

# -- 19 - separa em duas df: sem e com supervisores, pois no relatorios tem estruturas diferentes.. 
df_com_supervisor = df_detalhes[
    (df_detalhes['Nm supervisor'].notna()) & 
    (df_detalhes['Nm supervisor'] != 'SEM SUPERVISOR')
]
df_sem_supervisor = df_detalhes[
    df_detalhes['Nm supervisor'] == 'SEM SUPERVISOR'
]

# -- 20 - tranforma adta em string para evitar complicações na hora de gerar o relatorio
df_sem_supervisor['Dt. emissão'] = df_sem_supervisor['Dt. emissão'].dt.strftime("%Y-%m-%d")
df_com_supervisor['Dt. emissão'] = df_com_supervisor['Dt. emissão'].dt.strftime("%Y-%m-%d")





C:\Users\ADMIN\AppData\Local\Temp\ipykernel_12836\1636551535.py:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_comission_raw = pd.concat([df_comission_raw,df_nfs], axis=0)
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_12836\1636551535.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clientes_novos['cliente_nv'] = 20
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_12836\1636551535.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[r

### load(gera relatorios linha a linha e salva em todos os lugares) 09 - comissoes e resumo de comissoes 


In [ ]:
path_relatorio = fr'G:\Meu Drive\MASQUERADE\ADM-FINANCEIRO\RELATORIOS\COMISSOES\06-2025 em diante\relatorio_comissao_0{mes}-0{mes2}_{ano}.xlsx'
workbook = xlsxwriter.Workbook(path_relatorio)
worksheet = workbook.add_worksheet('Relatório')

# Formatações
titulo = workbook.add_format({
    'bold': True, 
    'bg_color': "#A7A7A7", 
    'border': 6,
    'align': 'center', 'valign': 'vcenter'
})
supervisor_format = workbook.add_format({
    'bold': True, 
    'bg_color': "#6680F0", 
    'border': 6,
    'align': 'center', 'valign': 'vcenter'
})
vendedor_format = workbook.add_format({
    'bold': True, 
    'bg_color': "#60A165", 
    'border': 6,
    'align': 'center', 'valign': 'vcenter'
})
cols = workbook.add_format({
    'border': 6, 
    'bg_color': "#92D397", 
    'align': 'center', 'valign': 'vcenter'
})
infos = workbook.add_format({
    'border': 2, 
    'align': 'center', 'valign': 'vcenter'
})

worksheet.merge_range('A1:H1', f"relatorio de comissão do mes {mes}/{2025} para pagar no mes {mes2}/{2025}", titulo)
linha = 3

# Supervisores
for supervisor in df_com_supervisor['Nm supervisor'].unique():
    worksheet.merge_range(linha, 0, linha, 2,  f"SUPERVISOR: {supervisor}", supervisor_format)
    linha += 2

    vendedores = df_com_supervisor[df_com_supervisor['Nm supervisor'] == supervisor]['Nm Vendedor'].unique()
    for vendedor in vendedores:
        worksheet.write(linha, 2, f"VENDEDOR: {vendedor}", vendedor_format)
        linha += 1

        headers = ["Data emissão", "NF", "Cliente", "Venda", "Faturado", "Comissão", "Cliente NV", "Total"]
        for col, header in enumerate(headers, start=0):
            worksheet.write(linha, col, header, cols)
        linha += 1

        vendas = df_com_supervisor[
            (df_com_supervisor['Nm supervisor'] == supervisor) &
            (df_com_supervisor['Nm Vendedor'] == vendedor)
        ]
        for _, row in vendas.iterrows():
            dados = [
                row['Dt. emissão'],
                f"NF {row['Nr. nota']}",
                row['Descrição'],
                row['Vl venda'],
                row['Vl Total Nota'],
                row['comissao'],
                row['cliente_nv'],
                row['total']
            ]
            for col, val in enumerate(dados, start=0):
                val = "" if pd.isna(val) else val
                worksheet.write(linha, col, val, infos)
            linha += 1

        subtot = subtotal_vendedor[
            (subtotal_vendedor['Nm supervisor'] == supervisor) &
            (subtotal_vendedor['Nm Vendedor'] == vendedor)
        ]
        if not subtot.empty:
            worksheet.write(linha, 2    , "TOTAL:", cols)
            worksheet.write(linha, 3, subtot['total_venda'].values[0], infos)
            worksheet.write(linha, 4, subtot['total_faturado'].values[0], infos)
            worksheet.write(linha, 5, subtot['total_comissao'].values[0], infos)
            worksheet.write(linha, 6, subtot['total_cliente_nv'].values[0], infos)
            worksheet.write(linha, 7, subtot['total_total'].values[0], infos)
            linha += 2

    resumo = resumo_supervisor.loc[supervisor]
    worksheet.write(linha, 2, "→ Comissão cliente NV:", cols)
    worksheet.write(linha, 3, resumo['total_cliente_nv'], infos)
    worksheet.write(linha + 1, 2, "→ 3% do grupo:", cols)
    worksheet.write(linha + 1, 3, resumo['comissao_extra_3pct'], infos)
    worksheet.write(linha + 2, 2, "→ TOTAL COMISSÃO FINAL:", cols)
    worksheet.write(linha + 2, 3, resumo['comissao_final'], infos)
    linha += 5

# Vendedores sem supervisor
for vendedor in df_sem_supervisor['Nm Vendedor'].unique():
    worksheet.write(linha, 2, f"VENDEDOR: {vendedor}", vendedor_format)
    linha += 1

    headers = ["Data emissão", "NF", "Cliente", "Venda", "Faturado", "Comissão", "Cliente NV", "Total"]
    for col, header in enumerate(headers, start=0):
        worksheet.write(linha, col, header, cols)
    linha += 1

    vendas = df_sem_supervisor[df_sem_supervisor['Nm Vendedor'] == vendedor]
    for _, row in vendas.iterrows():
        dados = [
            row['Dt. emissão'],
            f"NF {row['Nr. nota']}",
            row['Descrição'],
            row['Vl venda'],
            row['Vl Total Nota'],
            row['comissao'],
            row['cliente_nv'],
            row['total']
        ]
        for col, val in enumerate(dados, start=0):
            val = "" if pd.isna(val) else val
            worksheet.write(linha, col, val, infos)
        linha += 1

    subtot = subtotal_vendedor[subtotal_vendedor['Nm Vendedor'] == vendedor]
    if not subtot.empty:
        worksheet.write(linha, 2, "TOTAL:", cols)
        worksheet.write(linha, 3, subtot['total_venda'].values[0], infos)
        worksheet.write(linha, 4, subtot['total_faturado'].values[0], infos)
        worksheet.write(linha, 5, subtot['total_comissao'].values[0], infos)
        worksheet.write(linha, 6, subtot['total_cliente_nv'].values[0], infos)
        worksheet.write(linha, 7, subtot['total_total'].values[0], infos)
        linha += 2

# Fecha corretamente o arquivo
workbook.close()


In [88]:
# resumo
path_resumo = fr'G:\Meu Drive\MASQUERADE\ADM-FINANCEIRO\RELATORIOS\COMISSOES\06-2025 em diante\resumo_comissao_0{mes}-0{mes2}_{ano}.xlsx'
workbook = xlsxwriter.Workbook(path_resumo)
worksheet = workbook.add_worksheet('Resumo')

linha = 0

titulo = workbook.add_format({
    'bold': True, 
    'bg_color': "#A7A7A7", 
    'border': 6,
    'align': 'center', 'valign': 'vcenter'
})
cols = workbook.add_format({
    'border': 6, 
    'bold': True,
    'bg_color': "#72AC76", 
    'align': 'center', 'valign': 'vcenter'
})
infos = workbook.add_format({
    'border': 2, 
    'bg_color': "#94EEA3", 
    'align': 'center', 'valign': 'vcenter'
})


worksheet.merge_range('A1:G1', f'POSIÇÃO DE COMISSÕES DOS REPRESENTANTES 0{mes} PAGAMENTO EM 0{mes2}/{ano}', titulo)
linha += 1

headers = ['REPRESENTANTE',	'COMISSÃO',	'BANCO', 'AG.', 'CONTA', 'FAVORECIDO', 'CPF/CNPJ']
for col, header in enumerate(headers, start=0):
    worksheet.write(linha, col, header, cols)
linha += 1
for vendedor in df_detalhes['Nm Vendedor'].unique() :
    worksheet.write(linha,0,f'{vendedor}',infos)
    worksheet.write(linha,2,'PIX', infos)
    worksheet.write(linha,3,'', infos)
    worksheet.write(linha,4,'', infos)


    vendas = df_detalhes[df_detalhes['Nm Vendedor'] == vendedor]
    for _, row in vendas.iterrows():
        comisao = "" if pd.isna(row['comissao']) else row['comissao']
        favorecido = "" if pd.isna(row['FAVORECIDO']) else row['FAVORECIDO']
        chave = "" if pd.isna(row['CHAVE']) else row['CHAVE']
        
        worksheet.write(linha,1,comisao ,infos) # comis
        worksheet.write(linha,5,favorecido,infos ) # favorecio
        worksheet.write(linha,6, chave,infos ) # chave
    linha += 2
    
workbook.close()


### graficos 

# 01 - clientes novos por regiao por periodo ou vendedor



In [ ]:
''' mes / centro oeste / ...... / sul / total
total / ..... /  '''

In [68]:


# -- 1 - definindo dfs e colunas
df_clientes = df_clientes_masquerade[['Cd Uf','Nm Cidade','Dt Cadastro']]
df_estados_regiões = pd.DataFrame(estados_regiões)
df_novos = ['regiao', 'Nm cidade', 'Quant']

# -- 2 - definindo a região 
df_clientes = df_clientes.merge(df_estados_regiões[['Cd Uf', 'Região']],
                                on='Cd Uf', how='left' )

# -- 3 definindo periodo 
anos_meses = {2024: [1,2,3,4,5,6,8,9,10,11,12],
        2025: [1,2,3,4,5,6]}

# -- 4 - filtrando pelo peridodo
dados = [] # paera pord4er atribuir tudo depois 

for ano, meses in anos_meses.items():
    for mes in meses:
        primeiro_dia = datetime(ano, mes, 1)
        ultimo_dia = (primeiro_dia + relativedelta(months=1)) - relativedelta(days=1)
        
        df_novos_raw = df_clientes[
            (df_clientes['Dt Cadastro'] >= primeiro_dia) & 
            (df_clientes['Dt Cadastro'] <= ultimo_dia)
        ]

        for index, row in df_novos_raw.iterrows():
            dados.append({
                "Ano": ano,
                "Mês": mes,
                "Nm Cidade" : row['Nm Cidade'],
                "Região": row["Região"]
                 })

df_novos_raw = pd.DataFrame(dados)
df_novos_raw['Periodo'] = df_novos_raw['Ano'].astype(str) + '-' + df_novos_raw['Mês'].astype(str).str.zfill(2)

df_novos_2024 = df_novos_raw[df_novos_raw['Ano'] == 2024]
df_novos_2024.drop(columns=['Ano','Mês'], inplace=True)
df_novos_2024.groupby('Periodo')['Região'].count() .sum()

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_4828\604163167.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_novos_2024.drop(columns=['Ano','Mês'], inplace=True)


np.int64(121)

In [92]:
# Caminho do relatório
path_relatorio = r"G:\Meu Drive\MASQUERADE\FINANCEIRO\RELATORIOS\CLIENTES-NOVOS\clientes_novos_região_08-2024_06-2025.xlsx"
workbook = xlsxwriter.Workbook(path_relatorio)
worksheet = workbook.add_worksheet('Relatório')

# Linha inicial
linha = 0

# Estilos
titulo = workbook.add_format({
    'bold': True, 
    'bg_color': "#A7A7A7", 
    'border': 6,
    'align': 'center', 'valign': 'vcenter'
})
cols = workbook.add_format({
    'border': 6, 
    'bold': True,
    'bg_color': "#72AC76", 
    'align': 'center', 'valign': 'vcenter'
})
infos = workbook.add_format({
    'border': 2, 
    'align': 'center', 'valign': 'vcenter'
})

# Cabeçalho principal
worksheet.merge_range("A1:H1", "RELATÓRIO DE CLIENTES NOVOS POR REGIÃO - 2024 - 2025", titulo)
linha += 2

# ---- Preparação dos dados

# Pivot
df_pivot = df_novos_raw.pivot_table(index='Periodo', columns='Região', aggfunc='size', fill_value=0)

# Garante todas as regiões
regioes = ['Centro-Oeste', 'Nordeste', 'Norte', 'Sudeste', 'Sul']
for reg in regioes:
    if reg not in df_pivot.columns:
        df_pivot[reg] = 0

df_pivot = df_pivot[regioes]
df_pivot['Total'] = df_pivot.sum(axis=1)
df_pivot.reset_index(inplace=True)

# Filtra por ano
df_2024 = df_pivot[df_pivot['Periodo'].str.startswith('2024')]
df_2025 = df_pivot[df_pivot['Periodo'].str.startswith('2025')]

headers = ['Período'] + regioes + ['Total']

# Função para escrever DataFrame
def escrever_df(worksheet, df, linha, format_info):
    for _, row in df.iterrows():
        for col, colname in enumerate(df.columns):
            worksheet.write(linha, col, row[colname], format_info)
        linha += 1
    return linha

# ---------- BLOCO 2024 ----------
worksheet.write(linha, 0, "2024", titulo)
linha += 1

for col, header in enumerate(headers):
    worksheet.write(linha, col, header, cols)
linha += 1

linha = escrever_df(worksheet, df_2024, linha, infos)

totais_2024 = df_2024[regioes + ['Total']].sum()
worksheet.write(linha, 0, 'Total 2024', cols)
for col, val in enumerate(totais_2024, start=1):
    worksheet.write(linha, col, val, infos)
linha += 2

# ---------- BLOCO 2025 ----------
worksheet.write(linha, 0, "2025", titulo)
linha += 1

for col, header in enumerate(headers):
    worksheet.write(linha, col, header, cols)
linha += 1

linha = escrever_df(worksheet, df_2025, linha, infos)

totais_2025 = df_2025[regioes + ['Total']].sum()
worksheet.write(linha, 0, 'Total 2025', cols)
for col, val in enumerate(totais_2025, start=1):
    worksheet.write(linha, col, val, infos)

# Finaliza o arquivo
workbook.close()
